This notebook explores dependency parsing by identifying the actions and objects that are characteristically associated with male and female characters.

In [1]:
import spacy, math
from collections import Counter
import operator

In [2]:
nlp = spacy.load('en_core_web_sm')

We'll run seven novels by Jane Austen through spacy (this will take a few minutes).

In [3]:
filenames=["../data/fiction/emma.txt", "../data/fiction/lady_susan.txt", "../data/fiction/mansfield_park.txt", "../data/fiction/northanger_abbey.txt", "../data/fiction/persuasion.txt", "../data/fiction/pride.txt", "../data/fiction/sense_and_sensibility.txt"]
all_tokens=[]
for filename in filenames:
    print(filename)
    data=open(filename, encoding="utf-8").read()
    tokens=nlp(data)
    all_tokens.extend(tokens)

../data/fiction/emma.txt
../data/fiction/lady_susan.txt
../data/fiction/mansfield_park.txt
../data/fiction/northanger_abbey.txt
../data/fiction/persuasion.txt
../data/fiction/pride.txt
../data/fiction/sense_and_sensibility.txt


In [4]:
print (len(all_tokens))

972808


In [5]:
def test(maleCounter, femaleCounter, display=25):
    
    """ Function that takes two Counter objects as inputs and prints out a ranked list of terms
    more characteristic of the first counter than the second.  Here we'll use log-odds
    with an uninformative prior (from Monroe et al 2008, "Fightin Words", eqn. 22) as our metric.
    
    """
    
    vocab=dict(maleCounter) 
    vocab.update(dict(femaleCounter))
    maleSum=sum(maleCounter.values())
    femaleSum=sum(femaleCounter.values())

    ranks={}
    alpha=0.01
    alphaV=len(vocab)*alpha
        
    for word in vocab:
        
        log_odds_ratio=math.log( (maleCounter[word] + alpha) / (maleSum+alphaV-maleCounter[word]-alpha) ) - math.log( (femaleCounter[word] + alpha) / (femaleSum+alphaV-femaleCounter[word]-alpha) )
        variance=1./(maleCounter[word] + alpha) + 1./(femaleCounter[word] + alpha)
        
        ranks[word]=log_odds_ratio/math.sqrt(variance)

    sorted_x = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)
    
    print("Most male:")
    for k,v in sorted_x[:display]:
        print("%.3f\t%s" % (v,k))
    
    print("\nMost female:")
    for k,v in reversed(sorted_x[-display:]):
        print("%.3f\t%s" % (v,k))

Spacy uses the [ClearNLP dependency labels](https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md), which are very close to the Stanford typed dependencies.  See the [Stanford dependencies manual](https://nlp.stanford.edu/software/dependencies_manual.pdf) for more information about each tag.  Parse information is contained in the spacy token object; see the following for which attributes encode the token text, idx (position in sentence), part of speech, and dependency relation.  The syntactic head for a token is another token given in `token.head` (where all of those same token attributes are accessible).

In [6]:
testDoc=nlp("He started his car.")
for token in testDoc:
    print("%s\t%s\t%s\t%s\t%s\t%s\t%s" % (token.text, token.idx, token.tag_, token.dep_, token.head.text, token.head.idx, token.head.tag_))


He	0	PRP	nsubj	started	3	VBD
started	3	VBD	ROOT	started	3	VBD
his	11	PRP$	poss	car	15	NN
car	15	NN	dobj	started	3	VBD
.	18	.	punct	started	3	VBD


Q1: Find the verbs that men are more characteristically the *subject* of than women.  Feel free to only consider subjects that are "he" and "she" pronouns.  This function should return two Counter objects (`maleCounter` and `femaleCounter`) which counts the number of times a given verb has "he" (`maleCounter`) and "she" (`femaleCounter`) as its syntactic subject.

In [15]:
def count_subjects():
    maleCounter=Counter()
    femaleCounter=Counter()

    for token in all_tokens:
        if token.dep_ == "nsubj":
            if token.text.lower() == "he":
                maleCounter[token.head.text]+=1
            elif token.text.lower() == "she":
                femaleCounter[token.head.text]+=1
    
    return maleCounter, femaleCounter

In [16]:
male, female=count_subjects()
test(male, female, display=10)

Most male:
6.887	is
5.237	replied
4.911	come
4.718	told
4.692	came
4.399	said
4.048	seemed
3.508	left
3.495	done
3.465	comes

Most female:
-7.654	felt
-5.324	saw
-4.336	knew
-4.206	heard
-4.019	found
-3.599	cried
-3.281	was
-2.954	thought
-2.935	help
-2.925	had


Q2: Find the verbs that men are more characteristically the *object* of than women.  Feel free to only consider objects that are "him" and "here" pronouns.  This function should return two Counter objects (`maleCounter` and `femaleCounter`) which counts the number of times a given verb has "he" (`maleCounter`) and "she" (`femaleCounter`) as its syntactic direct object. 

In [17]:
def count_objects():
    maleCounter=Counter()
    femaleCounter=Counter()

    for token in all_tokens:
        if token.dep_ == "dobj":
            if token.text.lower() == "him":
                maleCounter[token.head.text]+=1
            elif token.text.lower() == "her":
                femaleCounter[token.head.text]+=1
    
    return maleCounter, femaleCounter

In [18]:
male, female=count_objects()
test(male, female, display=10)

Most male:
3.754	like
3.598	seen
3.445	seeing
2.944	know
2.841	meet
2.344	liked
2.340	see
2.215	call
2.082	do
2.059	sent

Most female:
-3.889	left
-2.671	struck
-2.493	convinced
-2.388	attended
-2.023	gave
-1.977	told
-1.904	obliged
-1.850	joined
-1.758	prevented
-1.727	amuse


Q3: Find the objects that are *possessed* more frequently by men than women.  Feel free to only consider possessors that are "his" and "her" pronouns.   This function should return two Counter objects (`maleCounter` and `femaleCounter`) which counts the number of times a given term is possessed by "he" (`maleCounter`) and "she" (`femaleCounter`).

In [19]:
def count_possessions():
    maleCounter=Counter()
    femaleCounter=Counter()

    for token in all_tokens:
        if token.dep_ == "poss":
            if token.text.lower() == "his":
                maleCounter[token.head.text]+=1
            elif token.text.lower() == "her":
                femaleCounter[token.head.text]+=1
    
    return maleCounter, femaleCounter

In [20]:
male, female=count_possessions()
test(male, female, display=10)

Most male:
4.430	sisters
4.369	attentions
4.177	house
4.150	being
4.118	return
3.921	name
3.555	horse
3.472	attachment
3.453	son
3.347	character

Most female:
-7.129	mother
-6.184	sister
-4.995	aunt
-4.568	eyes
-4.130	uncle
-3.700	heart
-3.677	spirits
-3.564	room
-3.179	thoughts
-3.120	hand


Q4: Find the actions that are men do *to women* more frequently than women do *to men*.  Feel free to only consider subjects and objects that are "she"/"he"/"her"/"him" pronouns.   This function should return two Counter objects (`maleCounter` and `femaleCounter`) which counts the number of times a given verb has "he" as the subject and "her" as the object (`maleCounter`) and "she" as the subject and "him" as the object (`femaleCounter`).

In [21]:
def count_SVO_tuples():
    maleCounter=Counter()
    femaleCounter=Counter()

    dobj_verbs={}
    nsubj_verbs={}

    for token in all_tokens:
        if token.dep_ == "dobj":
            if token.text.lower() == "him" or token.text.lower() == "her":
                head=token.head
                if head not in dobj_verbs:
                    dobj_verbs[head]=[]
                dobj_verbs[head].append(token)
                
                
        if token.dep_ == "nsubj":
            if token.text.lower() == "he" or token.text.lower() == "she":
                head=token.head
                if head not in nsubj_verbs:
                    nsubj_verbs[head]=[]
                nsubj_verbs[head].append(token)

    for head_verb in dobj_verbs:
        if head_verb in nsubj_verbs:
            
            for subjectToken in nsubj_verbs[head_verb]:
                for objectToken in dobj_verbs[head_verb]:
                    if subjectToken.text.lower() == "he" and objectToken.text.lower() == "her":
                        maleCounter[head_verb.text]+=1
                    elif subjectToken.text.lower() == "she" and objectToken.text.lower() == "him":
                        femaleCounter[head_verb.text]+=1

                
    return maleCounter, femaleCounter

In [22]:
male, female=count_SVO_tuples()
test(male, female, display=10)

Most male:
2.919	told
1.911	loved
1.778	left
1.344	joined
1.235	asked
1.047	admire
0.804	assured
0.652	called
0.652	sought
0.652	distinguished

Most female:
-2.434	seen
-1.587	have
-1.219	saw
-0.870	found
-0.693	see
-0.674	liked
-0.613	accept
-0.613	refused
-0.591	accepted
-0.591	like
